alexnet cg


In [ ]:
import numpy as np
import pandas as pd
import cv2
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import os
import xlsxwriter
import zipfile
import glob
from tqdm import tqdm  # For progress bar

# Function to convert temperature file to image
def temperature_to_image(temp_file_path):
    """
    Convert temperature file (CSV) to thermal image.
    """
    # Read temperature values from CSV
    try:
        temp_data = pd.read_csv(temp_file_path).values
    except FileNotFoundError:
        print(f"Error: CSV file not found at {temp_file_path}")
        return None, None
    except pd.errors.EmptyDataError:
        print(f"Error: CSV file is empty at {temp_file_path}")
        return None, None
    except Exception as e:
        print(f"Error reading CSV file {temp_file_path}: {str(e)}")
        return None, None

    # Check if temp_data is empty
    if temp_data.size == 0:
        print(f"Warning: No temperature data found in {temp_file_path}. Returning None image.")
        return None, None

    # Normalize temperature values between 0 and 255
    normalized_data = cv2.normalize(temp_data, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Apply color mapping to get thermal visualization
    thermal_image = cv2.applyColorMap(normalized_data, cv2.COLORMAP_JET)

    return thermal_image, normalized_data

# Function to preprocess thermal image
def preprocess_thermal_image(thermal_image):
    """
    Preprocess thermal image for DL feature extraction.
    Includes normalization and CLAHE contrast enhancement.
    """
    if thermal_image is None:
        print("Warning: Received None image for preprocessing. Returning None.")
        return None

    # Convert to grayscale
    gray_image = cv2.cvtColor(thermal_image, cv2.COLOR_BGR2GRAY)

    # Apply Contrast Limited Adaptive Histogram Equalization (CLAHE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(gray_image)

    # Convert back to RGB format for AlexNet input
    preprocessed_image = cv2.cvtColor(enhanced_image, cv2.COLOR_GRAY2RGB)

    return preprocessed_image

# Initialize AlexNet model once to avoid reloading for each file
def initialize_model():
    """
    Initialize and prepare the AlexNet model for feature extraction.
    """
    try:
        model = models.alexnet(pretrained=True)
        model.classifier = torch.nn.Sequential(*list(model.classifier.children())[:-1])  # Remove final layer
        model.eval()  # Set model to evaluation mode
    except Exception as e:
        print(f"Error initializing AlexNet model: {str(e)}")
        return None, None

    # Define preprocessing pipeline
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    return model, preprocess

# Function to extract features using pre-initialized AlexNet
def extract_dl_features(image, model, preprocess):
    """
    Extract deep learning features using AlexNet.
    """
    if image is None:
        print("Warning: Received None image for feature extraction. Returning None.")
        return None

    # Convert to PIL image and preprocess
    pil_image = Image.fromarray(image)
    input_tensor = preprocess(pil_image).unsqueeze(0)  # Add batch dimension

    # Extract features
    try:
        with torch.no_grad():
            features = model(input_tensor)
    except Exception as e:
        print(f"Error extracting features: {str(e)}")
        return None

    return features.squeeze().numpy()  # Return as 1D numpy array

# Function to export features to Excel
def export_features_to_excel(features_list, file_names, output_file):
    """
    Export extracted features to an Excel file.

    Args:
        features_list (list): List of feature arrays
        file_names (list): List of file names corresponding to the features
        output_file (str): Path to save the Excel file
    """
    workbook = xlsxwriter.Workbook(output_file)
    worksheet = workbook.add_worksheet()

    # Write header with file names
    worksheet.write(0, 0, "File Name")
    if features_list and features_list[0] is not None:
        for col_idx in range(len(features_list[0])):
            worksheet.write(0, col_idx + 1, f"Feature_{col_idx+1}")

        # Write features row by row, with file names in the first column
        for row_idx, (features, file_name) in enumerate(zip(features_list, file_names)):
            worksheet.write(row_idx + 1, 0, file_name)
            if features is not None:
                for col_idx, value in enumerate(features):
                    worksheet.write(row_idx + 1, col_idx + 1, float(value))
            else:
                print(f"Warning: No features to write for {file_name}.")
    else:
        print("Warning: No features available to export.")

    workbook.close()
    print(f"Features successfully exported to {output_file}")

# Function to process zip file containing CSV files
def process_zip_file(zip_file_path, output_excel):
    """
    Extract CSV files from a zip file, process them to extract features,
    and save all features to a single Excel file.

    Args:
        zip_file_path (str): Path to the zip file containing CSV files
        output_excel (str): Path to save the extracted features
    """
    # Create temporary directory to extract files
    temp_dir = "temp_extracted_files"
    os.makedirs(temp_dir, exist_ok=True)

    # Initialize model and preprocessor once
    model, preprocess = initialize_model()
    if model is None or preprocess is None:
        print("Error: Model initialization failed. Cannot continue processing.")
        return [], []

    try:
        # Extract all files from the zip file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        # Find all CSV files in the extracted directory
        csv_files = glob.glob(os.path.join(temp_dir, "**", "*.csv"), recursive=True)

        if not csv_files:
            print(f"No CSV files found in {zip_file_path}")
            return [], []

        print(f"Found {len(csv_files)} CSV files to process")

        # Extract features from all CSV files
        all_features = []
        file_names = []

        for i, csv_file in enumerate(tqdm(csv_files, desc="Processing files")):
            try:
                # Step 1: Convert temperature file to image
                thermal_image, _ = temperature_to_image(csv_file)

                if thermal_image is None:
                    print(f"Warning: Thermal image conversion failed for {csv_file}. Skipping.")
                    all_features.append(None)  # Append None to maintain order
                    file_names.append(os.path.basename(csv_file))
                    continue

                # Step 2: Preprocess thermal image
                preprocessed_image = preprocess_thermal_image(thermal_image)

                if preprocessed_image is None:
                    print(f"Warning: Preprocessing failed for {csv_file}. Skipping.")
                    all_features.append(None)  # Append None to maintain order
                    file_names.append(os.path.basename(csv_file))
                    continue

                # Step 3: Extract DL features using AlexNet
                features = extract_dl_features(preprocessed_image, model, preprocess)

                if features is None:
                    print(f"Warning: Feature extraction failed for {csv_file}. Skipping.")
                    all_features.append(None)  # Append None to maintain order
                    file_names.append(os.path.basename(csv_file))
                    continue

                all_features.append(features)
                file_names.append(os.path.basename(csv_file))

            except Exception as e:
                print(f"Error processing {csv_file}: {str(e)}")
                all_features.append(None)  # Append None to maintain order
                file_names.append(os.path.basename(csv_file))

        # Step 4: Export all features to a single Excel file
        if all_features:
            export_features_to_excel(all_features, file_names, output_excel)
            print(f"All features have been exported to {output_excel}")
        else:
            print("No features were extracted. Excel file not created.")

        return all_features, file_names

    finally:
        # Clean up the temporary directory
        import shutil
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

# For Google Colab: Upload zip file function
def upload_and_process_zip():
    """
    Upload a zip file in Google Colab and process all CSV files inside it.
    """
    from google.colab import files
    uploaded = files.upload()

    if not uploaded:
        print("No file was uploaded")
        return

    zip_file_path = next(iter(uploaded.keys()))
    output_excel = "alexnet_features.xlsx"

    # Process the uploaded zip file
    process_zip_file(zip_file_path, output_excel)

    # Download the resulting Excel file
    if os.path.exists(output_excel):
        files.download(output_excel)

# Main execution
if __name__ == "__main__":
    # Check if running in Google Colab
    try:
        import google.colab
        print("Running in Google Colab. Please upload your zip file containing CSV files.")
        upload_and_process_zip()
    except ImportError:
        # For local execution or Jupyter notebook
        print("Running locally. Please specify the path to your zip file.")

        # Path to the zip file containing CSV files
        zip_file_path = "/content/DM Group 2.zip"  # Replace with your zip file path

        # Output Excel file path for extracted features
        output_excel = "alexnet_features.xlsx"

        # Process zip file and save features to Excel
        process_zip_file(zip_file_path, output_excel)


Running in Google Colab. Please upload your zip file containing CSV files.


Saving DM Group 2.zip to DM Group 2 (1).zip


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Found 244 CSV files to process


Processing files: 100%|██████████| 244/244 [00:19<00:00, 12.76it/s]


Features successfully exported to alexnet_features.xlsx
All features have been exported to alexnet_features.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

efficient net


In [ ]:
import numpy as np
import pandas as pd
import cv2
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import os
import xlsxwriter
import zipfile
import glob
from tqdm import tqdm  # For progress bar

# Function to convert temperature file to image
def temperature_to_image(temp_file_path):
    """
    Convert temperature file (CSV) to thermal image.
    """
    # Read temperature values from CSV
    try:
        temp_data = pd.read_csv(temp_file_path).values
    except FileNotFoundError:
        print(f"Error: CSV file not found at {temp_file_path}")
        return None, None
    except pd.errors.EmptyDataError:
        print(f"Error: CSV file is empty at {temp_file_path}")
        return None, None
    except Exception as e:
        print(f"Error reading CSV file {temp_file_path}: {str(e)}")
        return None, None

    # Check if temp_data is empty
    if temp_data.size == 0:
        print(f"Warning: No temperature data found in {temp_file_path}. Returning None image.")
        return None, None

    # Normalize temperature values between 0 and 255
    normalized_data = cv2.normalize(temp_data, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Apply color mapping to get thermal visualization
    thermal_image = cv2.applyColorMap(normalized_data, cv2.COLORMAP_JET)

    return thermal_image, normalized_data

# Function to preprocess thermal image
def preprocess_thermal_image(thermal_image):
    """
    Preprocess thermal image for DL feature extraction.
    Includes normalization and CLAHE contrast enhancement.
    """
    if thermal_image is None:
        print("Warning: Received None image for preprocessing. Returning None.")
        return None

    # Convert to grayscale
    gray_image = cv2.cvtColor(thermal_image, cv2.COLOR_BGR2GRAY)

    # Apply Contrast Limited Adaptive Histogram Equalization (CLAHE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(gray_image)

    # Convert back to RGB format for EfficientNet input
    preprocessed_image = cv2.cvtColor(enhanced_image, cv2.COLOR_GRAY2RGB)

    return preprocessed_image

# Initialize EfficientNetB0 model once to avoid reloading for each file
def initialize_model():
    """
    Initialize and prepare the EfficientNetB0 model for feature extraction.
    """
    try:
        model = models.efficientnet_b0(pretrained=True)
        model.classifier = torch.nn.Identity()  # Remove the classifier head
        model.eval()  # Set model to evaluation mode
    except Exception as e:
        print(f"Error initializing EfficientNetB0 model: {str(e)}")
        return None, None

    # Define preprocessing pipeline
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    return model, preprocess

# Function to extract features using pre-initialized EfficientNetB0
def extract_dl_features(image, model, preprocess):
    """
    Extract deep learning features using EfficientNetB0.
    """
    if image is None:
        print("Warning: Received None image for feature extraction. Returning None.")
        return None

    # Convert to PIL image and preprocess
    pil_image = Image.fromarray(image)
    input_tensor = preprocess(pil_image).unsqueeze(0)  # Add batch dimension

    # Extract features
    try:
        with torch.no_grad():
            features = model(input_tensor)
    except Exception as e:
        print(f"Error extracting features: {str(e)}")
        return None

    return features.squeeze().numpy()  # Return as 1D numpy array

# Function to export features to Excel
def export_features_to_excel(features_list, file_names, output_file):
    """
    Export extracted features to an Excel file.

    Args:
        features_list (list): List of feature arrays
        file_names (list): List of file names corresponding to the features
        output_file (str): Path to save the Excel file
    """
    workbook = xlsxwriter.Workbook(output_file)
    worksheet = workbook.add_worksheet()

    # Write header with file names
    worksheet.write(0, 0, "File Name")
    if features_list and features_list[0] is not None:
        for col_idx in range(len(features_list[0])):
            worksheet.write(0, col_idx + 1, f"Feature_{col_idx+1}")

        # Write features row by row, with file names in the first column
        for row_idx, (features, file_name) in enumerate(zip(features_list, file_names)):
            worksheet.write(row_idx + 1, 0, file_name)
            if features is not None:
                for col_idx, value in enumerate(features):
                    worksheet.write(row_idx + 1, col_idx + 1, float(value))
            else:
                print(f"Warning: No features to write for {file_name}.")
    else:
        print("Warning: No features available to export.")

    workbook.close()
    print(f"Features successfully exported to {output_file}")

# Function to process zip file containing CSV files
def process_zip_file(zip_file_path, output_excel):
    """
    Extract CSV files from a zip file, process them to extract features,
    and save all features to a single Excel file.

    Args:
        zip_file_path (str): Path to the zip file containing CSV files
        output_excel (str): Path to save the extracted features
    """
    # Create temporary directory to extract files
    temp_dir = "temp_extracted_files"
    os.makedirs(temp_dir, exist_ok=True)

    # Initialize model and preprocessor once
    model, preprocess = initialize_model()
    if model is None or preprocess is None:
        print("Error: Model initialization failed. Cannot continue processing.")
        return [], []

    try:
        # Extract all files from the zip file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        # Find all CSV files in the extracted directory
        csv_files = glob.glob(os.path.join(temp_dir, "**", "*.csv"), recursive=True)

        if not csv_files:
            print(f"No CSV files found in {zip_file_path}")
            return [], []

        print(f"Found {len(csv_files)} CSV files to process")

        # Extract features from all CSV files
        all_features = []
        file_names = []

        for i, csv_file in enumerate(tqdm(csv_files, desc="Processing files")):
            try:
                # Step 1: Convert temperature file to image
                thermal_image, _ = temperature_to_image(csv_file)

                if thermal_image is None:
                    print(f"Warning: Thermal image conversion failed for {csv_file}. Skipping.")
                    all_features.append(None)  # Append None to maintain order
                    file_names.append(os.path.basename(csv_file))
                    continue

                # Step 2: Preprocess thermal image
                preprocessed_image = preprocess_thermal_image(thermal_image)

                if preprocessed_image is None:
                    print(f"Warning: Preprocessing failed for {csv_file}. Skipping.")
                    all_features.append(None)  # Append None to maintain order
                    file_names.append(os.path.basename(csv_file))
                    continue

                # Step 3: Extract DL features using EfficientNetB0
                features = extract_dl_features(preprocessed_image, model, preprocess)

                if features is None:
                    print(f"Warning: Feature extraction failed for {csv_file}. Skipping.")
                    all_features.append(None)  # Append None to maintain order
                    file_names.append(os.path.basename(csv_file))
                    continue

                all_features.append(features)
                file_names.append(os.path.basename(csv_file))

            except Exception as e:
                print(f"Error processing {csv_file}: {str(e)}")
                all_features.append(None)  # Append None to maintain order
                file_names.append(os.path.basename(csv_file))

        # Step 4: Export all features to a single Excel file
        if all_features:
            export_features_to_excel(all_features, file_names, output_excel)
            print(f"All features have been exported to {output_excel}")
        else:
            print("No features were extracted. Excel file not created.")

        return all_features, file_names

    finally:
        # Clean up the temporary directory
        import shutil
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

# For Google Colab: Upload zip file function
def upload_and_process_zip():
    """
    Upload a zip file in Google Colab and process all CSV files inside it.
    """
    from google.colab import files
    uploaded = files.upload()

    if not uploaded:
        print("No file was uploaded")
        return

    zip_file_path = next(iter(uploaded.keys()))
    output_excel = "efficientnetb0_features.xlsx"

    # Process the uploaded zip file
    process_zip_file(zip_file_path, output_excel)

    # Download the resulting Excel file
    if os.path.exists(output_excel):
        files.download(output_excel)

# Main execution
if __name__ == "__main__":
    # Check if running in Google Colab
    try:
        import google.colab
        print("Running in Google Colab. Please upload your zip file containing CSV files.")
        upload_and_process_zip()
    except ImportError:
        # For local execution or Jupyter notebook
        print("Running locally. Please specify the path to your zip file.")

        # Path to the zip file containing CSV files
        zip_file_path = "/content/DM Group 2.zip"  # Replace with your zip file path

        # Output Excel file path for extracted features
        output_excel = "efficientnetb0_features.xlsx"

        # Process zip file and save features to Excel
        process_zip_file(zip_file_path, output_excel)


Running in Google Colab. Please upload your zip file containing CSV files.


Saving DM Group 2.zip to DM Group 2 (1).zip


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Found 244 CSV files to process


Processing files: 100%|██████████| 244/244 [00:19<00:00, 12.62it/s]


Features successfully exported to efficientnetb0_features.xlsx
All features have been exported to efficientnetb0_features.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

resnet50

In [ ]:
import numpy as np
import pandas as pd
import cv2
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import os
import xlsxwriter
import zipfile
import glob
from tqdm import tqdm  # For progress bar

# Function to convert temperature file to image
def temperature_to_image(temp_file_path):
    """
    Convert temperature file (CSV) to thermal image.
    """
    try:
        temp_data = pd.read_csv(temp_file_path).values
    except FileNotFoundError:
        print(f"Error: CSV file not found at {temp_file_path}")
        return None, None
    except pd.errors.EmptyDataError:
        print(f"Error: CSV file is empty at {temp_file_path}")
        return None, None
    except Exception as e:
        print(f"Error reading CSV file {temp_file_path}: {str(e)}")
        return None, None

    if temp_data.size == 0:
        print(f"Warning: No temperature data found in {temp_file_path}. Returning None image.")
        return None, None

    normalized_data = cv2.normalize(temp_data, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    thermal_image = cv2.applyColorMap(normalized_data, cv2.COLORMAP_JET)

    return thermal_image, normalized_data

# Function to preprocess thermal image
def preprocess_thermal_image(thermal_image):
    """
    Preprocess thermal image for DL feature extraction.
    Includes normalization and CLAHE contrast enhancement.
    """
    if thermal_image is None:
        print("Warning: Received None image for preprocessing. Returning None.")
        return None

    gray_image = cv2.cvtColor(thermal_image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(gray_image)
    preprocessed_image = cv2.cvtColor(enhanced_image, cv2.COLOR_GRAY2RGB)

    return preprocessed_image

# Initialize ResNet50 model once to avoid reloading for each file
def initialize_model():
    """
    Initialize and prepare the ResNet50 model for feature extraction.
    """
    try:
        model = models.resnet50(pretrained=True)
        model.fc = torch.nn.Identity()  # Remove final fully connected layer
        model.eval()  # Set model to evaluation mode
    except Exception as e:
        print(f"Error initializing ResNet50 model: {str(e)}")
        return None, None

    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    return model, preprocess

# Function to extract features using pre-initialized ResNet50
def extract_dl_features(image, model, preprocess):
    """
    Extract deep learning features using ResNet50.
    """
    if image is None:
        print("Warning: Received None image for feature extraction. Returning None.")
        return None

    pil_image = Image.fromarray(image)
    input_tensor = preprocess(pil_image).unsqueeze(0)

    try:
        with torch.no_grad():
            features = model(input_tensor)
    except Exception as e:
        print(f"Error extracting features: {str(e)}")
        return None

    return features.squeeze().numpy()

# Function to export features to Excel
def export_features_to_excel(features_list, file_names, output_file):
    """
    Export extracted features to an Excel file.

    Args:
        features_list (list): List of feature arrays
        file_names (list): List of file names corresponding to the features
        output_file (str): Path to save the Excel file
    """
    workbook = xlsxwriter.Workbook(output_file)
    worksheet = workbook.add_worksheet()

    worksheet.write(0, 0, "File Name")

    if features_list and features_list[0] is not None:
        for col_idx in range(len(features_list[0])):
            worksheet.write(0, col_idx + 1, f"Feature_{col_idx+1}")

        for row_idx, (features, file_name) in enumerate(zip(features_list, file_names)):
            worksheet.write(row_idx + 1, 0, file_name)
            if features is not None:
                for col_idx, value in enumerate(features):
                    worksheet.write(row_idx + 1, col_idx + 1, float(value))
            else:
                print(f"Warning: No features to write for {file_name}.")

    workbook.close()
    print(f"Features successfully exported to {output_file}")

# Function to process zip file containing CSV files
def process_zip_file(zip_file_path, output_excel):
    """
    Extract CSV files from a zip file and process them to extract features.

    Args:
        zip_file_path (str): Path to the zip file containing CSV files
        output_excel (str): Path to save extracted features in Excel format.
    """
    temp_dir = "temp_extracted_files"
    os.makedirs(temp_dir, exist_ok=True)

    model, preprocess = initialize_model()

    if model is None or preprocess is None:
        print("Error: Model initialization failed.")
        return [], []

    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        csv_files = glob.glob(os.path.join(temp_dir, "**", "*.csv"), recursive=True)

        if not csv_files:
            print(f"No CSV files found in {zip_file_path}")
            return [], []

        all_features = []
        file_names = []

        for csv_file in tqdm(csv_files, desc="Processing files"):
            try:
                thermal_image, _ = temperature_to_image(csv_file)

                if thermal_image is None:
                    all_features.append(None)
                    file_names.append(os.path.basename(csv_file))
                    continue

                preprocessed_image = preprocess_thermal_image(thermal_image)

                if preprocessed_image is None:
                    all_features.append(None)
                    file_names.append(os.path.basename(csv_file))
                    continue

                features = extract_dl_features(preprocessed_image, model, preprocess)

                if features is None:
                    all_features.append(None)
                    file_names.append(os.path.basename(csv_file))
                    continue

                all_features.append(features)
                file_names.append(os.path.basename(csv_file))

            except Exception as e:
                print(f"Error processing {csv_file}: {str(e)}")
                all_features.append(None)
                file_names.append(os.path.basename(csv_file))

        if all_features:
            export_features_to_excel(all_features, file_names, output_excel)
            print(f"All features have been exported to {output_excel}")

        return all_features, file_names

    finally:
        import shutil
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

if __name__ == "__main__":

    zip_file_path = "/content/DM Group 2.zip"  # Replace with your zip file path

    output_excel = "resnet50_features.xlsx"

    process_zip_file(zip_file_path, output_excel)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Processing files: 100%|██████████| 244/244 [01:03<00:00,  3.83it/s]


Features successfully exported to resnet50_features.xlsx
All features have been exported to resnet50_features.xlsx
